In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
#import mplfinance as mpf

In [3]:
from LayerModel5 import *

In [4]:
from frame import *

In [5]:
import os
import psutil
import gc

# 显示当前 python 程序占用的内存大小
def show_memory_info(hint):
    pid = os.getpid()
    p = psutil.Process(pid)
   
    info = p.memory_full_info()
    memory = info.uss / 1024. / 1024
    print('{} memory used: {} MB'.format(hint, memory))


In [6]:
show_memory_info('initial')

initial memory used: 76.16796875 MB


## ricequant csv导入

In [7]:
dt_df = pd.read_csv('data_csv\\0601882.XSHG.csv',index_col=0,parse_dates=True)
stockID ='0601882.XSHG.csv'

In [8]:
dt_df['TmIdx'] = list(range(0,dt_df.shape[0]))
dt_df['tm'] =dt_df.index

In [9]:
dt_df.head()

,open,high,low,close,total_turnover,volume,TmIdx,tm
datetime,,,,,,,,
2020-02-06 09:35:00,6.5134,6.5134,6.4239,6.4338,105672.0,16200.0,0,2020-02-06 09:35:00
2020-02-06 09:40:00,6.4139,6.4239,6.3941,6.4239,264564.0,41000.0,1,2020-02-06 09:40:00
2020-02-06 09:45:00,6.4239,6.4438,6.4139,6.4438,151274.0,23400.0,2,2020-02-06 09:45:00
2020-02-06 09:50:00,6.4438,6.4438,6.4239,6.4338,198778.0,30700.0,3,2020-02-06 09:50:00
2020-02-06 09:55:00,6.4338,6.4438,6.4338,6.4338,76379.0,11800.0,4,2020-02-06 09:55:00


## get data list

In [10]:
dt = dt_df[['open','high','low','close','TmIdx','tm']].values.tolist()
#idx = dt_df[['datetime','TmIdx']]
#idx['date_str'] = idx.loc[:,'datetime'].apply(lambda x:x.strftime('%m-%d %H:%M'))

In [11]:
show_memory_info('initial')

initial memory used: 79.65234375 MB


### Get 属性数据

In [12]:
def main():
    global dt,m,ef
    layer = m.layer
    
    Event.L = [[] for i in list(range(layer+1))]
    #filepath = 'event_config.json'
    #ef = EventFactory(filepath)
    
    m.update(dt[-1])
    for k in dt[1:]:
        m.update(k)
        Stick.L[-1].update(k)  
        
        # update crt Lv1 and add new Lv1
        TrendLv1.L[-1].updateEndP(k)
        for i in list(range(layer)):
            if len(Event.L[i]) > 0:
                print('TmIdx:{},[{}]'.format(k[4],[x for x in Event.L[i]]))
                for event in Event.L[i][:]:
                    actions = ef.play(event)
                    for a in actions:
                        print(a)
                        if a != '':
                            eval(a)
             
    return None

In [13]:
filepath = 'event_config.json'
ef = EventFactory(filepath)

In [14]:
m = Market(stockID, 4)
Stick.mm = m
Trend.m = m
Trend.ef = ef
Pair.m = m
PairChain.m = m
PairChain.ef = ef
CenterStrict.m = m
CenterStrict.ef = ef
#Signal001.m = m
#Signal001.ef = ef

In [15]:
Stick.L = m.Lv0_L
Stick('init', k_bar=dt[0])

Stick(drt:0, lv:0, Point(0, 6.5134, 0))

In [16]:
TrendLv1 = type('TrendLv1', (Trend,), {'L':m.Lv1_L, 'ML':Stick.L, 'level':1}) 
TrendLv1('init', k_bar=dt[0], mp=[0])
TrendLv2 = type('TrendLv2', (Trend,), {'L':m.Lv2_L, 'ML':TrendLv1.L, 'level':2}) 
TrendLv2('init', k_bar=dt[0], mp=[0])
TrendLv3 = type('TrendLv3', (Trend,), {'L':m.Lv3_L, 'ML':TrendLv2.L, 'level':3}) 
TrendLv3('init', k_bar=dt[0], mp=[0])

Trend3(0, 0, Point(0, 6.5134, 0))

In [17]:

Center0 = type('Center0', (CenterStrict,), {'ML': m.findList('st', 0), 'L': m.findList('center', 0), 'openL':[],  'level': 0})
Center0()
Center1 = type('Center1', (CenterStrict,), {'ML': m.findList('st', 1), 'L': m.findList('center', 1), 'openL':[],  'level': 1})
Center1()
Center2 = type('Center2', (CenterStrict,), {'ML': m.findList('st', 2), 'L': m.findList('center', 2), 'openL':[],  'level': 2})
Center2()
Center3 = type('Center3', (CenterStrict,), {'ML': m.findList('st', 3), 'L': m.findList('center', 3), 'openL':[],  'level': 3})
Center3()

Center3(0, 1, 0, 0)

In [18]:
PairLv0 = type('PairLv0', (Pair,), {'ML': m.findList('st', 0), 'L': m.findList('pair', 0), 'level': 0})
PairLv0(0)
PairLv1 = type('PairLv1', (Pair,), {'ML': m.findList('st', 1), 'L': m.findList('pair', 1), 'level': 1})
PairLv1(0)
PairLv2 = type('PairLv2', (Pair,), {'ML': m.findList('st', 2), 'L': m.findList('pair', 2), 'level': 2})
PairLv2(0)
PairLv3 = type('PairLv3', (Pair,), {'ML': m.findList('st', 3), 'L': m.findList('pair', 3), 'level': 3})
PairLv3(0)
PairChainLv0 = PairChain(0, 'PairChainLv0')
PairChainLv1 = PairChain(1,'PairChainLv1')
PairChainLv2 = PairChain(2, 'PairChainLv2')
PairChainLv3 = PairChain(3, 'PairChainLv3')

In [19]:
SIG_overlap.m, SIG_overlap.ef = m, ef
SIG_overlap()
actions = []
level = 2
actions.append({ 'level_num': level, 
            'obj_name': 'TrendLv' +str(int(level)),
            'event_name': 'NEW',
            'obj_p': 'SIG_overlap',
            'method': 'any_opp',
            'param': str(level)  })
for ac in actions:
    ef.regAction(**ac)

In [20]:
SIG_CCrawl.m, SIG_CCrawl.ef = m, ef
SIG_CCrawl([2])
actions = []
level = 2
actions.append({ 'level_num': level, 
            'obj_name': 'Center' +str(int(level)),
            'event_name': 'NEW10',
            'obj_p': 'SIG_CCrawl',
            'method': 'any_opp',
            'param': str(level)  })
for ac in actions:
    ef.regAction(**ac)

In [ ]:
show_memory_info('initial')

In [ ]:
Signal001.L = m.SIG_L
Signal001()

In [ ]:
Position.m = m
Position.L = m.position

In [ ]:
class Pattern01Lv0(Pattern01):
    def __init__(self,cursor):   
        super().__init__()
        self.changeL(m.findList('pattern01', 0))
        self.L.append(self)
        self.level = 0
        self.chain_layer = cursor[0]
        self.changeML(m.findList('pairchain', 0))
        self.cursor = cursor
        self.changeW(2)      


In [ ]:
class Pattern01Lv1(Pattern01):
    def __init__(self,cursor):   
        super().__init__()
        self.changeL(m.findList('pattern01', 1))
        self.L.append(self)
        self.level = 1
        self.chain_layer = cursor[0]
        self.changeML(m.findList('pairchain', 1))
        self.cursor = cursor
        self.changeW(2)      

In [ ]:
class Pattern01Lv2(Pattern01):
    def __init__(self,cursor):   
        super().__init__()
        self.changeL(m.findList('pattern01', 2))
        self.L.append(self)
        self.level = 2
        self.chain_layer = cursor[0]
        self.changeML(m.findList('pairchain', 2))
        self.cursor = cursor
        self.changeW(2)      

In [ ]:
'''PatternPair.m = m
Pattern01Lv0([0,0])
Pattern01Lv0([1,0])
Pattern01Lv0([2,0])
m.PT01Lv0_L[0].regAction()
Pattern01Lv1([0,0])
Pattern01Lv1([1,0])
Pattern01Lv1([2,0])
m.PT01Lv1_L[0].regAction()
Pattern01Lv2([0,0])
Pattern01Lv2([1,0])
Pattern01Lv2([2,0])
m.PT01Lv2_L[0].regAction()
'''

In [ ]:
m.__dict__

In [ ]:
ef.event_config

In [21]:
time_start=time.time()
main()
time_end=time.time()
print('totally cost：',time_end-time_start)

TmIdx:5,[[<frame.Event object at 0x0000000017A873C8>]]
m.Lv1_L[-1].updatePeakP(k_bar=m.dt[-1])
TmIdx:13,[[<frame.Event object at 0x000000001766EA20>]]
m.Lv1_L[-1].update2()
m.PCLv0_L[0].update2()
Center0.updateAll()
TmIdx:14,[[<frame.Event object at 0x000000001766EE48>]]
TmIdx:16,[[<frame.Event object at 0x0000000017A8B7B8>]]
m.Lv1_L[-1].update2()
m.PCLv0_L[0].update2()
Center0.updateAll()
TmIdx:16,[[<frame.Event object at 0x000000001766EA20>]]
m.Lv2_L[-1].updatePeakP(k_bar=m.dt[-1])
TmIdx:17,[[<frame.Event object at 0x0000000017A8B240>]]
TmIdx:23,[[<frame.Event object at 0x0000000017A8BA20>]]
m.PCLv0_L[0].updateLastSt()
TmIdx:24,[[<frame.Event object at 0x0000000017A8BAC8>]]
m.PCLv0_L[0].updateLastSt()
TmIdx:25,[[<frame.Event object at 0x0000000017A8B780>]]
m.PCLv0_L[0].updateLastSt()
TmIdx:26,[[<frame.Event object at 0x0000000017A8B2E8>]]
m.Lv1_L[-1].updatePeakP(k_bar=m.dt[-1])
TmIdx:34,[[<frame.Event object at 0x0000000017A8BFD0>]]
m.Lv1_L[-1].update2()
m.PCLv0_L[0].update2()
Center

TmIdx:857,[[<frame.Event object at 0x0000000017B2EDD8>]]
m.Lv2_L[-1].updatePeakP(k_bar=m.dt[-1])
TmIdx:858,[[<frame.Event object at 0x0000000017B362B0>]]
m.CLv0_L[0].newCenter(obj="pair", level=0, i=-1)
TmIdx:864,[[<frame.Event object at 0x0000000017B36438>]]
m.Lv1_L[-1].update2()
m.PCLv0_L[0].update2()
Center0.updateAll()
TmIdx:867,[[<frame.Event object at 0x0000000017B36630>]]
m.Lv1_L[-1].update2()
m.PCLv0_L[0].update2()
Center0.updateAll()
TmIdx:868,[[<frame.Event object at 0x0000000017B367B8>]]
TmIdx:878,[[<frame.Event object at 0x0000000017B36A58>]]
m.PCLv0_L[0].updateLastSt()
TmIdx:881,[[<frame.Event object at 0x0000000017B36630>]]
m.Lv1_L[-1].updatePeakP(k_bar=m.dt[-1])
TmIdx:884,[[<frame.Event object at 0x0000000017B36A90>]]
m.PCLv0_L[0].updateLastSt()
TmIdx:886,[[<frame.Event object at 0x0000000017B36DA0>]]
m.Lv1_L[-1].updatePeakP(k_bar=m.dt[-1])
TmIdx:896,[[<frame.Event object at 0x0000000017AE1160>]]
m.Lv1_L[-1].update2()
m.PCLv0_L[0].update2()
Center0.updateAll()
TmIdx:897,

Center0.updateAll()
TmIdx:1635,[[<frame.Event object at 0x0000000017BC14E0>]]
m.Lv2_L[-1].update2()
m.PCLv1_L[0].update2()
Center1.updateAll()
m.CLv0_L[0].newCenter(obj="st", level=1, i=-1)
TmIdx:1636,[[<frame.Event object at 0x0000000017BC19B0>]]
m.CLv0_L[0].newCenter(obj="pair", level=0, i=-1)
TmIdx:1636,[[<frame.Event object at 0x0000000017BC1940>]]
m.CLv1_L[0].newCenter(obj="pair", level=1, i=-1)
TmIdx:1638,[[<frame.Event object at 0x0000000017BC1AC8>]]
m.Lv1_L[-1].update2()
m.PCLv0_L[0].update2()
Center0.updateAll()
TmIdx:1638,[[<frame.Event object at 0x0000000017BC1940>]]
m.Lv2_L[-1].update2()
m.PCLv1_L[0].update2()
Center1.updateAll()
m.CLv0_L[0].newCenter(obj="st", level=1, i=-1)
TmIdx:1639,[[<frame.Event object at 0x0000000017BC1DA0>]]
TmIdx:1643,[[<frame.Event object at 0x0000000017BC1F60>]]
m.Lv1_L[-1].update2()
m.PCLv0_L[0].update2()
Center0.updateAll()
TmIdx:1643,[[<frame.Event object at 0x0000000017BC1940>]]
m.Lv2_L[-1].update2()
m.PCLv1_L[0].update2()
Center1.updateAll()

TmIdx:2242,[[<frame.Event object at 0x0000000017645B00>]]
m.CLv0_L[0].newCenter(obj="pair", level=0, i=-1)
TmIdx:2244,[[<frame.Event object at 0x00000000176458D0>]]
m.Lv1_L[-1].update2()
m.PCLv0_L[0].update2()
Center0.updateAll()
TmIdx:2244,[[<frame.Event object at 0x0000000017645B00>]]
m.Lv2_L[-1].update2()
m.PCLv1_L[0].update2()
Center1.updateAll()
m.CLv0_L[0].newCenter(obj="st", level=1, i=-1)
TmIdx:2245,[[<frame.Event object at 0x000000001767E5F8>]]
m.CLv0_L[0].newCenter(obj="pair", level=0, i=-1)
TmIdx:2245,[[<frame.Event object at 0x0000000017680160>]]
TmIdx:2247,[[<frame.Event object at 0x0000000017645B00>]]
m.PCLv0_L[0].updateLastSt()
TmIdx:2249,[[<frame.Event object at 0x000000001767E5F8>]]
m.PCLv0_L[0].updateLastSt()
TmIdx:2252,[[<frame.Event object at 0x0000000017662630>]]
m.Lv1_L[-1].updatePeakP(k_bar=m.dt[-1])
TmIdx:2258,[[<frame.Event object at 0x00000000176622B0>]]
m.Lv1_L[-1].update2()
m.PCLv0_L[0].update2()
Center0.updateAll()
TmIdx:2258,[[<frame.Event object at 0x0000

Center0.updateAll()
TmIdx:2881,[[<frame.Event object at 0x0000000017C99828>]]
m.Lv1_L[-1].update2()
m.PCLv0_L[0].update2()
Center0.updateAll()
TmIdx:2888,[[<frame.Event object at 0x0000000017C99A90>]]
m.PCLv0_L[0].updateLastSt()
TmIdx:2890,[[<frame.Event object at 0x0000000017C99BE0>]]
m.Lv1_L[-1].updatePeakP(k_bar=m.dt[-1])
TmIdx:2896,[[<frame.Event object at 0x0000000017C99E10>]]
m.Lv1_L[-1].update2()
m.PCLv0_L[0].update2()
Center0.updateAll()
TmIdx:2908,[[<frame.Event object at 0x0000000017C452E8>]]
m.Lv1_L[-1].update2()
m.PCLv0_L[0].update2()
Center0.updateAll()
TmIdx:2922,[[<frame.Event object at 0x0000000017C457F0>]]
m.Lv1_L[-1].update2()
m.PCLv0_L[0].update2()
Center0.updateAll()
TmIdx:2926,[[<frame.Event object at 0x0000000017C45A58>]]
m.Lv1_L[-1].update2()
m.PCLv0_L[0].update2()
Center0.updateAll()
TmIdx:2928,[[<frame.Event object at 0x0000000017C45BA8>]]
m.PCLv0_L[0].updateLastSt()
TmIdx:2929,[[<frame.Event object at 0x0000000017C45C88>]]
m.Lv1_L[-1].updatePeakP(k_bar=m.dt[-1

m.PCLv1_L[0].update2()
Center1.updateAll()
m.CLv0_L[0].newCenter(obj="st", level=1, i=-1)
TmIdx:3447,[[<frame.Event object at 0x0000000017CF0908>]]
m.Lv3_L[-1].update2()
m.PCLv2_L[0].update2()
Center2.updateAll()
m.CLv1_L[0].newCenter(obj="st", level=2, i=-1)
SIG_overlap.any_opp(2)
SIG_CCrawl.established()
TmIdx:3448,[[<frame.Event object at 0x0000000017CF0C18>, <frame.Event object at 0x0000000017CF0AC8>]]
m.CLv0_L[0].newCenter(obj="pair", level=0, i=-1)
m.PCLv0_L[0].updateLastSt()
TmIdx:3448,[[<frame.Event object at 0x0000000017CF0D30>]]
TmIdx:3449,[[<frame.Event object at 0x0000000017CF0EB8>]]
m.PCLv0_L[0].updateLastSt()
TmIdx:3450,[[<frame.Event object at 0x0000000017CF0F28>]]
m.PCLv0_L[0].updateLastSt()
TmIdx:3451,[[<frame.Event object at 0x0000000017CF0F60>]]
m.PCLv0_L[0].updateLastSt()
TmIdx:3452,[[<frame.Event object at 0x0000000017CF0F28>]]
m.PCLv0_L[0].updateLastSt()
TmIdx:3453,[[<frame.Event object at 0x0000000017CF80B8>]]
m.PCLv0_L[0].updateLastSt()
TmIdx:3454,[[<frame.Event

TmIdx:4087,[[<frame.Event object at 0x0000000017D44DA0>]]
m.Lv2_L[-1].updatePeakP(k_bar=m.dt[-1])
TmIdx:4088,[[<frame.Event object at 0x0000000017D5E2B0>]]
m.CLv0_L[0].newCenter(obj="pair", level=0, i=-1)
TmIdx:4094,[[<frame.Event object at 0x0000000017D5E4E0>]]
m.Lv1_L[-1].update2()
m.PCLv0_L[0].update2()
Center0.updateAll()
TmIdx:4095,[[<frame.Event object at 0x0000000017D5E668>]]
TmIdx:4098,[[<frame.Event object at 0x0000000017D5E7B8>]]
m.Lv1_L[-1].update2()
m.PCLv0_L[0].update2()
Center0.updateAll()
TmIdx:4098,[[<frame.Event object at 0x0000000017D5E4E0>]]
m.Lv2_L[-1].update2()
m.PCLv1_L[0].update2()
Center1.updateAll()
m.CLv0_L[0].newCenter(obj="st", level=1, i=-1)
TmIdx:4098,[[<frame.Event object at 0x0000000017D5E7B8>]]
m.Lv3_L[-1].update2()
m.PCLv2_L[0].update2()
Center2.updateAll()
m.CLv1_L[0].newCenter(obj="st", level=2, i=-1)
SIG_overlap.any_opp(2)
SIG_CCrawl.established()
TmIdx:4098,[[<frame.Event object at 0x0000000017D5E4E0>]]

TmIdx:4099,[[<frame.Event object at 0x000000

TmIdx:4578,[[<frame.Event object at 0x0000000017DB10F0>]]
m.Lv2_L[-1].updatePeakP(k_bar=m.dt[-1])
TmIdx:4579,[[<frame.Event object at 0x0000000017DB1710>]]
TmIdx:4582,[[<frame.Event object at 0x0000000017DB1898>]]
m.Lv1_L[-1].update2()
m.PCLv0_L[0].update2()
Center0.updateAll()
TmIdx:4583,[[<frame.Event object at 0x0000000017DB1748>]]
m.CLv0_L[0].newCenter(obj="pair", level=0, i=-1)
TmIdx:4588,[[<frame.Event object at 0x0000000017DB1B70>]]
m.Lv1_L[-1].update2()
m.PCLv0_L[0].update2()
Center0.updateAll()
TmIdx:4588,[[<frame.Event object at 0x0000000017DB1748>]]
m.Lv2_L[-1].updatePeakP(k_bar=m.dt[-1])
TmIdx:4589,[[<frame.Event object at 0x0000000017DB1DA0>, <frame.Event object at 0x0000000017DB1C88>]]
m.CLv0_L[0].newCenter(obj="pair", level=0, i=-1)
m.PCLv0_L[0].updateLastSt()
TmIdx:4591,[[<frame.Event object at 0x0000000017DB1DA0>]]
m.Lv1_L[-1].updatePeakP(k_bar=m.dt[-1])
TmIdx:4598,[[<frame.Event object at 0x0000000017D880F0>]]
m.Lv1_L[-1].update2()
m.PCLv0_L[0].update2()
Center0.updat

m.PCLv2_L[0].update2()
Center2.updateAll()
m.CLv1_L[0].newCenter(obj="st", level=2, i=-1)
SIG_overlap.any_opp(2)
SIG_CCrawl.established()
TmIdx:5041,[[<frame.Event object at 0x0000000017E03DA0>]]

TmIdx:5042,[[<frame.Event object at 0x0000000017E05390>]]
m.CLv0_L[0].newCenter(obj="pair", level=0, i=-1)
TmIdx:5042,[[<frame.Event object at 0x0000000017E054E0>]]
m.CLv1_L[0].newCenter(obj="pair", level=1, i=-1)
TmIdx:5042,[[<frame.Event object at 0x0000000017E056A0>, <frame.Event object at 0x0000000017E05748>]]
SIG_CCrawl.any_opp(2)
TmIdx:5044,[[<frame.Event object at 0x0000000017E05390>]]
m.Lv1_L[-1].update2()
m.PCLv0_L[0].update2()
Center0.updateAll()
TmIdx:5044,[[<frame.Event object at 0x0000000017E05748>]]
m.Lv2_L[-1].updatePeakP(k_bar=m.dt[-1])
TmIdx:5047,[[<frame.Event object at 0x0000000017E05940>]]
m.Lv1_L[-1].update2()
m.PCLv0_L[0].update2()
Center0.updateAll()
TmIdx:5052,[[<frame.Event object at 0x0000000017E05BE0>]]
m.Lv1_L[-1].update2()
m.PCLv0_L[0].update2()
Center0.updateAll(

TmIdx:5419,[[<frame.Event object at 0x0000000017E5A278>]]
m.Lv1_L[-1].update2()
m.PCLv0_L[0].update2()
Center0.updateAll()
TmIdx:5421,[[<frame.Event object at 0x0000000017E5A400>]]
m.PCLv0_L[0].updateLastSt()
TmIdx:5424,[[<frame.Event object at 0x0000000017E5A278>]]
m.PCLv0_L[0].updateLastSt()
TmIdx:5426,[[<frame.Event object at 0x0000000017E5A5F8>]]
m.Lv1_L[-1].updatePeakP(k_bar=m.dt[-1])
TmIdx:5430,[[<frame.Event object at 0x0000000017E5A828>]]
m.Lv1_L[-1].update2()
m.PCLv0_L[0].update2()
Center0.updateAll()
TmIdx:5431,[[<frame.Event object at 0x0000000017E5A9B0>]]
m.CLv0_L[0].newCenter(obj="pair", level=0, i=-1)
TmIdx:5435,[[<frame.Event object at 0x0000000017E5AB38>]]
m.Lv1_L[-1].update2()
m.PCLv0_L[0].update2()
Center0.updateAll()
TmIdx:5435,[[<frame.Event object at 0x0000000017E5A9B0>]]
m.Lv2_L[-1].update2()
m.PCLv1_L[0].update2()
Center1.updateAll()
m.CLv0_L[0].newCenter(obj="st", level=1, i=-1)
TmIdx:5435,[[<frame.Event object at 0x0000000017E5AB38>]]
m.Lv3_L[-1].update2()
m.PC

In [22]:
[[sig.TmS, sig.drt, sig.status, sig.open_tm] for sig in SIG_CCrawl.L]

[[4896, 1, 1, '2020-07-20 11']]

In [ ]:
SIG_CCrawl.remark

In [ ]:
show_memory_info('Init')
gc.collect()
show_memory_info('finished')

In [ ]:
[[sig.TmS,sig.drt] for sig in SIG_CCrawl.L if sig.status == 1]

In [ ]:
[sig.TmS for sig in SIG_CCrawl.L]

In [ ]:
[[cc.TmS, cc.is_main] for cc in Center2.L if cc.is_main >0]

In [ ]:
dd = [[st.start.TmIdx, st.start.V, st.peak.V, st.drt] for st in m.Lv2_L]
lv2_df = pd.DataFrame(dd, columns=['TmIdx', 'start', 'peak','drt'])
lv2_df.to_excel('aaa.xlsx')

In [ ]:
dd = [[cc.TmS,cc.is_main, cc.st_idxL] for cc in m.CLv2_L]
cc2_df = pd.DataFrame(dd, columns=['TmIdx', 'is_main',''])
cc2_df.to_excel('ccc.xlsx')

In [ ]:
for i in ll[-3::-1]:
    print(i)

In [ ]:
len(SIG_overlapMv.L) #59  # is_wiped 之后只剩8个  #cross Center 收紧到0.25 55个
# 中信只有37

In [ ]:
ll = m.findList('center', 0)
centerL = []
for cc in ll[::-1]:
    if cc.TmE < 10323:
        break
    if cc.TmS < 10323 <= cc.TmE:
        centerL.append(cc)
centerL

In [ ]:
time_start=time.time()
lll = [l.TmS for l in m.CLv0_L[::-1] if l.TmS < 10323 <l.TmE]
time_end=time.time()
print('totally cost：',time_end-time_start)

In [ ]:
lll

In [ ]:
list0 = [[st.start.TmIdx,st.peak.TmIdx] for st in m.Lv0_L if 10200<st.start.TmIdx<10600]
list1 = [st.start.TmIdx for st in m.Lv1_L if 10200<st.start.TmIdx<10600]
ll =[]
for s,p in list0:
    if s in list1 and p in list1:
        ll.append(s)

In [ ]:
[[st.start.TmIdx,st.start.V,st.peak.TmIdx,st.peak.V,st.ML[st.mp[st.pp]].start, st.mp[0]] for st in m.Lv1_L if 10200<st.start.TmIdx<10600]

In [ ]:
[[st.start.TmIdx,st.start.V,st.peak.TmIdx,st.peak.V] for st in m.Lv0_L if 10382<st.start.TmIdx<10465]

In [ ]:
dd99 = [[st.start.TmIdx, st.peak.V, st.ML[st.mp[st.pp-1]].peak.V, st.pp] for st in m.Lv1_L if st.peak.V != st.ML[st.mp[st.pp]].start.V]
len(dd99)

In [ ]:
dd99

In [ ]:
for i,st in enumerate(m.Lv1_L):
    if st.start.TmIdx == 8685:
        print(i)
        break

In [ ]:
SIG_overlapMv.Fremark

In [ ]:
st = m.Lv1_L[270]
st.pp, st.mp

In [ ]:
m.Lv1_L[320].remark

In [ ]:
dd = [[s.TmS, s.drt,s.HTmS, s.HP.drt,s.HP_cc,s.HP_st_len, s.HL_limit,
 dt_df.iloc[int(s.TmS)].name,dt_df.iloc[int(s.TmS)].close,
  dt_df.iloc[int(s.TmS):int(s.TmS)+480]['low'].min(), dt_df.iloc[int(s.TmS):int(s.TmS)+480]['high'].max()] for s in m.SIG_L[2:]]

In [ ]:
df_out = pd.DataFrame(dd)
df_out.columns = ['TmS', 'sig_drt', 'HTmS','HP.drt','s.HP_cc','s.HP_st_len', 'ccHL', 'Tm', 'V', 'V-min', 'V-max']

In [ ]:
df_out.to_excel('SIG.xlsx')

In [ ]:
m.Lv1_L[94]

In [ ]:
dt_df.iloc[2664].name

In [ ]:
dd=[]
dd = [[st.start.TmIdx, st.start.V, st.drt,st.peak.TmIdx, st.end.TmIdx,st.peak.TmIdx - st.start.TmIdx] for st in m.Lv2_L]

In [ ]:
dd=[]
dd = [[p.TmS, p.drt, p.S, p.P] for p in m.PCLv2_L[0].cL[0]]
dd

In [ ]:
rangeL, rangeH = dt_df[i-w:i+w].TmIdx.values[0], dt_df[i-w:i+w].TmIdx.values[-1]

In [ ]:
rangeL, rangeH

In [ ]:
m.PCLv0_L[0].cL[0][0].__dict__

In [ ]:
df_out = pd.DataFrame(dd)
df_out.columns = ['TmS', 'start', 'drt', 'TmP', 'TmE', 'T']
df_out['A'] = -df_out.start.diff(-1)

In [ ]:
for key in m.__dict__:
    print(key)
    if key[-2:] =='_L':
        ll = m.__dict__[key]
        for i in range(len(ll))[::-1]:
            del m.__dict__[key][i]

In [ ]:
for key in m.obj_list:
    print(key)
    if key[-2:] =='_L':
        for oo in m.obj_list[key]:
            del oo
    else:
        for level in m.obj_list[key]:
            print(type(level))
            if isinstance(level, list):
                for i in range(len(level)):
                    print(level[i])
                    del level[i]
gc.collect()

In [ ]:
m.obj_list.keys()

In [ ]:
del m.obj_list['st'][0][0]

In [ ]:
len( m.obj_list['st'][0]), m.obj_list['st'][0]

In [ ]:
import gc

In [ ]:
gc.collect()

In [ ]:
len(m.Lv0_L),m.Lv0_L

In [ ]:
len(Stick.L),TrendLv2.L

In [ ]:
Event.L[0]

### 快照

In [ ]:
dt_df['close'].plot(figsize=(2000/72,800/72))

### Matplotlib 

In [ ]:
# prepare data
y = dt_df['TmIdx'].values.tolist()

In [ ]:
x = dt_df['close'].values.tolist()

In [ ]:
dt = dt_df.iloc[:20]
dt = dt[['open', 'high', 'low','close']]
dt.index.name = 'Date'

In [ ]:
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
from matplotlib.lines import Line2D
from datetime import datetime

In [ ]:
def draw_pairs(ax, data_collection, facecolor='r', edgecolor='None', alpha=0.5, marker=False ):
    # draw pair stick
    for pp in data_collection:
        x = [pp.ML[pp.index[0]].start.TmIdx, pp.ML[pp.index[0]].peak.TmIdx]
        y = [pp.ML[pp.index[0]].start.V, pp.ML[pp.index[0]].peak.V]
        ax.add_line(Line2D(x,y, lw=2, c=facecolor, alpha=alpha))
        if marker:
            ax.text(x[0],y[0], pp.TmS, c= facecolor)
    
    # draw rectangle
    rect =[ Rectangle((pp.ML[pp.index[0]].peak.TmIdx, pp.ccHL[2]),
                      pp.ML[pp.index[-1]].peak.TmIdx - pp.ML[pp.index[0]].peak.TmIdx, pp.ccHL[3] - pp.ccHL[2])
           for pp in data_collection]
    pc = PatchCollection(rect, facecolor=facecolor, alpha=0.5, edgecolor=edgecolor)
    ax.add_collection(pc)  
    if marker:
        for pp in data_collection:
            ax.text(pp.ML[pp.index[0]].peak.TmIdx, pp.ccHL[2], pp.ccHL[2], c='r')
            ax.text(pp.ML[pp.index[0]].peak.TmIdx, pp.ccHL[3], pp.ccHL[3], c='r')
    
    return None


In [ ]:
def draw_trends(ax, data_collection, linecolor='crimson',  lw=0.5, alpha = None, marker=False ):
    x = [pp.start.TmIdx for pp in data_collection]
    y = [pp.start.V for pp in data_collection]
    ax.add_line(Line2D(x,y, linewidth=lw, color=linecolor, alpha=alpha))
    if marker:
        for st in data_collection:
            ax.text(st.start.TmIdx, st.start.V, st.start, c='navy')
    return None

In [ ]:
def draw_center(ax, data_collection, facecolor='r', edgecolor='None', lw=0.5, alpha = None, marker=False):
     # draw rectangle
    rect =[ Rectangle((cc.TmS, cc.L), cc.TmE - cc.TmS, cc.H - cc.L)
           for cc in data_collection]
    pc = PatchCollection(rect, facecolor=facecolor, alpha=0.2, edgecolor=edgecolor)
    ax.add_collection(pc) 
    if marker:
        for cc in data_collection:
            ax.text(cc.TmS, cc.H, cc.TmS, c='navy')
    return None

In [ ]:
# 画放大图
# Input: dt_df, 
#        [s_idx: e_idx], 
def detial_fig(dt_df, s_idx, e_idx):

    ## 拼接 x_label
    dt_df['label2'] = dt_df.index
    dt_df['label'] = dt_df['label2'].apply(lambda x: x.strftime("%m/%d %H"))
    dt_df['label'] = dt_df['TmIdx'].astype('str').str.cat(dt_df['label'], sep=' ')
    dt_df.drop(['label2'],axis=1, inplace=True)

    ## x_label 转换函数
    x_fmt = dt_df['label'].values.tolist()
    def format_date(x,pos=None):   
        x = int(x)
        if x<0 or x>len(x_fmt)-1:
            return ''
        return x_fmt[x]

    ## 底图数据 by [s_idx: e_idx]
    x = [s_idx, e_idx]
    y = [dt_df.iloc[s_idx:e_idx].high.max(), dt_df.iloc[s_idx:e_idx].low.min()]
    print(x,y)
    
    ## draw
    fig, ax = plt.subplots(figsize=(6000/300, 2400/300), dpi=300)
    
    #=== 底图设置
    ax.xaxis.set_major_formatter(ticker.FuncFormatter(format_date))
    ax.xaxis.set_major_locator(ticker.MultipleLocator(int((e_idx-s_idx)/10)))
    ax.plot(x, y, alpha=0)
    
    #=== 
    Lv0_list = [st for st in m.Lv0_L if s_idx<st.peak.TmIdx<=e_idx]
    draw_trends(ax, Lv0_list)
    Lv1_list = [st for st in m.Lv1_L if s_idx<st.peak.TmIdx<=e_idx]
    draw_trends(ax, Lv1_list, linecolor='b',alpha = 0.8, marker=True)
    Lv2_list = [st for st in m.Lv2_L if s_idx<st.peak.TmIdx<=e_idx]
    draw_trends(ax, Lv2_list, linecolor='indigo', lw=2, alpha = 0.8)
    
    center_list0 = [cc for cc in m.CLv0_L if cc.is_main==10 and s_idx<=cc.TmS and cc.TmE <=e_idx]
    draw_center(ax, center_list0)
    center_list1 = [cc for cc in m.CLv1_L if cc.is_main==10 and s_idx<=cc.TmS and cc.TmE <=e_idx]
    draw_center(ax, center_list1, facecolor='turquoise')
    
    center_list2 = [cc for cc in m.CLv2_L if s_idx<=cc.TmS and cc.TmE <=e_idx]
    #draw_center(ax, center_list2,  facecolor='indigo')
    
    sig_list =[ss for ss in SIG_overlapMv.L if s_idx<=ss.TmInit <=e_idx]
    for ss in sig_list:
        #ax.text(ss.TmInit, ss.sigV+1, ss.remark)
        ax.annotate(ss.remark.split(',')[0], xy=(ss.TmInit, ss.sigV), xytext=(ss.TmInit, ss.sigV+2),
                arrowprops=dict(facecolor='black',headwidth=2,headlength=2, shrink=0.05, width=0.5),
                )
    
    fig.autofmt_xdate()
    plt.savefig('图{}细节-{}-{}.png'.format(stockID,str(int(s_idx)), str(int(e_idx))))
    

In [ ]:
# 全景缩略图
# Input: dt_df, 
def overscan_fig(dt_df, stockID):

    ## 拼接 x_label
    dt_df['label2'] = dt_df.index
    dt_df['label'] = dt_df['label2'].apply(lambda x: x.strftime("%m/%d %H"))
    dt_df['label'] = dt_df['TmIdx'].astype('str').str.cat(dt_df['label'], sep=' ')
    dt_df.drop(['label2'],axis=1, inplace=True)
    
    x_data = dt_df['label'].values
    y_data = dt_df['close'].values

    ## x_label 转换函数
    x_fmt = dt_df['label'].values.tolist()
    def format_date(x,pos=None):   
        x = int(x)
        if x<0 or x>len(x_fmt)-1:
            return ''
        return x_fmt[x]
    
    ## draw
    fig, ax = plt.subplots(figsize=(6000/300, 2400/300), dpi=300)
    
    #=== 底图设置
    ax.xaxis.set_major_formatter(ticker.FuncFormatter(format_date))
    ax.xaxis.set_major_locator(ticker.MultipleLocator(int((len(x_data))/10)))
    ax.plot(x_data, y_data)
       
    fig.autofmt_xdate()
    plt.title(stockID)
    plt.savefig('图{}全景.png'.format(stockID))

In [ ]:
overscan_fig(dt_df, stockID)

In [ ]:
dt_df.info()

In [ ]:
s_idx, e_idx =14000, 19800
# 绘制全景图
# Input: dt_df, 
#        [s_idx: e_idx], 
#       对象列表？？ 

## 拼接 x_label
dt_df['label2'] = dt_df.index
dt_df['label'] = dt_df['label2'].apply(lambda x: x.strftime("%m/%d %H"))
dt_df['label'] = dt_df['TmIdx'].astype('str').str.cat(dt_df['label'], sep=' ')
dt_df.drop(['label2'],axis=1, inplace=True)

## x_label 转换函数
x_fmt = dt_df['label'].values.tolist()
def format_date(x,pos=None):   
    x = int(x)
    if x<0 or x>len(x_fmt)-1:
        return ''
    return x_fmt[x]

## 底图数据 by [s_idx: e_idx]
x = [s_idx, e_idx]
y = [dt_df.iloc[s_idx:e_idx].high.max(), dt_df.iloc[s_idx:e_idx].low.min()]
print(x,y)

## draw
fig, ax = plt.subplots(figsize=(6000/300, 2400/300), dpi=300)

#=== 底图设置
ax.xaxis.set_major_formatter(ticker.FuncFormatter(format_date))
ax.xaxis.set_major_locator(ticker.MultipleLocator(int((e_idx-s_idx)/10)))
ax.plot(x, y, alpha=0)

#=== 
Lv0_list = [st for st in m.Lv0_L if s_idx<st.peak.TmIdx<=e_idx]
draw_trends(ax, Lv0_list)
Lv1_list = [st for st in m.Lv1_L if s_idx<st.peak.TmIdx<=e_idx]
draw_trends(ax, Lv1_list, linecolor='b',alpha = 0.8, marker=False)
Lv2_list = [st for st in m.Lv2_L if s_idx<st.peak.TmIdx<=e_idx]
draw_trends(ax, Lv2_list, linecolor='indigo', lw=2, alpha = 0.8)

center_list0 = [cc for cc in m.CLv0_L if s_idx<=cc.TmS and cc.TmE <=e_idx]
draw_center(ax, center_list0)
center_list1 = [cc for cc in m.CLv1_L if cc.is_main==10 and s_idx<=cc.TmS and cc.TmE <=e_idx]
draw_center(ax, center_list1, facecolor='turquoise')

center_list2 = [cc for cc in m.CLv2_L if s_idx<=cc.TmS and cc.TmE <=e_idx]
#draw_center(ax, center_list2,  facecolor='indigo', marker=True)

sig_list =[ss for ss in SIG_overlapMv.L if s_idx<=ss.TmInit <=e_idx]
for ss in sig_list:
    #ax.text(ss.TmInit, ss.sigV+1, ss.remark)
    ax.annotate(ss.remark.split(',')[0], xy=(ss.TmInit, ss.sigV), xytext=(ss.TmInit, ss.sigV+2),
            arrowprops=dict(facecolor='black',headwidth=2,headlength=2, shrink=0.05, width=0.5),
            )
'''
pair_list = [pp for pp in m.PLv0_L 
             if pp.chain_layer == 0 and s_idx<pp.ML[pp.index[0]].peak.TmIdx<e_idx]
draw_pairs(ax, pair_list)
pair_list1 = [pp for pp in m.PLv1_L 
             if pp.chain_layer == 0 and s_idx<pp.ML[pp.index[0]].peak.TmIdx<e_idx]
draw_pairs(ax, pair_list1, facecolor='turquoise', alpha=0.8)
pair_list2 = [pp for pp in m.PLv2_L 
             if pp.chain_layer == 0 and s_idx<pp.ML[pp.index[0]].peak.TmIdx<e_idx]
draw_pairs(ax, pair_list2, facecolor='violet',marker=False, alpha=0.8)
'''

fig.autofmt_xdate()
plt.savefig('图{}-{}_{}Lv0.png'.format(stockID, str(int(s_idx)), str(int(e_idx))))

In [ ]:
# 绘制全景图
# Input: dt_df, 
#        [s_idx: e_idx], 
#       对象列表？？ 

## 拼接 x_label
dt_df['label2'] = dt_df.index
dt_df['label'] = dt_df['label2'].apply(lambda x: x.strftime("%m/%d %H"))
dt_df['label'] = dt_df['TmIdx'].astype('str').str.cat(dt_df['label'], sep=' ')
dt_df.drop(['label2'],axis=1, inplace=True)

## x_label 转换函数
x_fmt = dt_df['label'].values.tolist()
def format_date(x,pos=None):   
    x = int(x)
    if x<0 or x>len(x_fmt)-1:
        return ''
    return x_fmt[x]

## 底图数据 by [s_idx: e_idx]
x = [s_idx, e_idx]
y = [dt_df.iloc[s_idx:e_idx].high.max(), dt_df.iloc[s_idx:e_idx].low.min()]
print(x,y)

## draw
fig, ax = plt.subplots(figsize=(6000/300, 2400/300), dpi=300)

#=== 底图设置
ax.xaxis.set_major_formatter(ticker.FuncFormatter(format_date))
ax.xaxis.set_major_locator(ticker.MultipleLocator(int((e_idx-s_idx)/10)))
ax.plot(x, y, alpha=0)

#=== 
Lv0_list = [st for st in m.Lv0_L if s_idx<st.peak.TmIdx<=e_idx]
draw_trends(ax, Lv0_list)
Lv1_list = [st for st in m.Lv1_L if s_idx<st.peak.TmIdx<=e_idx]
draw_trends(ax, Lv1_list, linecolor='b',alpha = 0.8, marker=False)
Lv2_list = [st for st in m.Lv2_L if s_idx<st.peak.TmIdx<=e_idx]
draw_trends(ax, Lv2_list, linecolor='indigo', lw=2, alpha = 0.8)

center_list0 = [cc for cc in m.CLv0_L if cc.is_main==10 and s_idx<=cc.TmS and cc.TmE <=e_idx]
draw_center(ax, center_list0)
center_list1 = [cc for cc in m.CLv1_L if s_idx<=cc.TmS and cc.TmE <=e_idx]
draw_center(ax, center_list1, facecolor='turquoise')

center_list2 = [cc for cc in m.CLv2_L if s_idx<=cc.TmS and cc.TmE <=e_idx]
#draw_center(ax, center_list2,  facecolor='indigo', marker=True)

sig_list =[ss for ss in SIG_overlapMv.L if s_idx<=ss.TmInit <=e_idx]
for ss in sig_list:
    #ax.text(ss.TmInit, ss.sigV+1, ss.remark)
    ax.annotate(ss.remark.split(',')[0], xy=(ss.TmInit, ss.sigV), xytext=(ss.TmInit, ss.sigV+2),
            arrowprops=dict(facecolor='black',headwidth=2,headlength=2, shrink=0.05, width=0.5),
            )
'''
pair_list = [pp for pp in m.PLv0_L 
             if pp.chain_layer == 0 and s_idx<pp.ML[pp.index[0]].peak.TmIdx<e_idx]
draw_pairs(ax, pair_list)
pair_list1 = [pp for pp in m.PLv1_L 
             if pp.chain_layer == 0 and s_idx<pp.ML[pp.index[0]].peak.TmIdx<e_idx]
draw_pairs(ax, pair_list1, facecolor='turquoise', alpha=0.8)
pair_list2 = [pp for pp in m.PLv2_L 
             if pp.chain_layer == 0 and s_idx<pp.ML[pp.index[0]].peak.TmIdx<e_idx]
draw_pairs(ax, pair_list2, facecolor='violet',marker=False, alpha=0.8)
'''

fig.autofmt_xdate()
plt.savefig('图{}-{}_{}Lv1.png'.format(stockID, str(int(s_idx)), str(int(e_idx))))

In [ ]:
# 绘制全景图
# Input: dt_df, 
#        [s_idx: e_idx], 
#       对象列表？？ 

## 拼接 x_label
dt_df['label2'] = dt_df.index
dt_df['label'] = dt_df['label2'].apply(lambda x: x.strftime("%m/%d %H"))
dt_df['label'] = dt_df['TmIdx'].astype('str').str.cat(dt_df['label'], sep=' ')
dt_df.drop(['label2'],axis=1, inplace=True)

## x_label 转换函数
x_fmt = dt_df['label'].values.tolist()
def format_date(x,pos=None):   
    x = int(x)
    if x<0 or x>len(x_fmt)-1:
        return ''
    return x_fmt[x]

## 底图数据 by [s_idx: e_idx]
x = [s_idx, e_idx]
y = [dt_df.iloc[s_idx:e_idx].high.max(), dt_df.iloc[s_idx:e_idx].low.min()]
print(x,y)

## draw
fig, ax = plt.subplots(figsize=(6000/300, 2400/300), dpi=300)

#=== 底图设置
ax.xaxis.set_major_formatter(ticker.FuncFormatter(format_date))
ax.xaxis.set_major_locator(ticker.MultipleLocator(int((e_idx-s_idx)/10)))
ax.plot(x, y, alpha=0)

#=== 
Lv0_list = [st for st in m.Lv0_L if s_idx<st.peak.TmIdx<=e_idx]
draw_trends(ax, Lv0_list)
Lv1_list = [st for st in m.Lv1_L if s_idx<st.peak.TmIdx<=e_idx]
draw_trends(ax, Lv1_list, linecolor='b',alpha = 0.8, marker=False)
Lv2_list = [st for st in m.Lv2_L if s_idx<st.peak.TmIdx<=e_idx]
draw_trends(ax, Lv2_list, linecolor='indigo', lw=2, alpha = 0.8)

center_list0 = [cc for cc in m.CLv0_L if cc.is_main==10 and s_idx<=cc.TmS and cc.TmE <=e_idx]
draw_center(ax, center_list0)
center_list1 = [cc for cc in m.CLv1_L if cc.is_main==10 and s_idx<=cc.TmS and cc.TmE <=e_idx]
draw_center(ax, center_list1, facecolor='turquoise')

center_list2 = [cc for cc in m.CLv2_L if s_idx<=cc.TmS and cc.TmE <=e_idx]
draw_center(ax, center_list2,  facecolor='indigo', marker=True)

sig_list =[ss for ss in SIG_overlapMv.L if s_idx<=ss.TmInit <=e_idx]
for ss in sig_list:
    #ax.text(ss.TmInit, ss.sigV+1, ss.remark)
    ax.annotate(ss.remark.split(',')[0], xy=(ss.TmInit, ss.sigV), xytext=(ss.TmInit, ss.sigV+2),
            arrowprops=dict(facecolor='black',headwidth=2,headlength=2, shrink=0.05, width=0.5),
            )
'''
pair_list = [pp for pp in m.PLv0_L 
             if pp.chain_layer == 0 and s_idx<pp.ML[pp.index[0]].peak.TmIdx<e_idx]
draw_pairs(ax, pair_list)
pair_list1 = [pp for pp in m.PLv1_L 
             if pp.chain_layer == 0 and s_idx<pp.ML[pp.index[0]].peak.TmIdx<e_idx]
draw_pairs(ax, pair_list1, facecolor='turquoise', alpha=0.8)
pair_list2 = [pp for pp in m.PLv2_L 
             if pp.chain_layer == 0 and s_idx<pp.ML[pp.index[0]].peak.TmIdx<e_idx]
draw_pairs(ax, pair_list2, facecolor='violet',marker=False, alpha=0.8)
'''

fig.autofmt_xdate()
plt.savefig('图{}-{}_{}Lv2.png'.format(stockID, str(int(s_idx)), str(int(e_idx))))

In [ ]:
c = 5600
r = 100
s_idx, e_idx = c-r, c+r

detial_fig(dt_df, s_idx, e_idx)

In [ ]:
for s in SIG_overlapMv.L:
    if s.lv_TmS == 10323:
        print(s.remark)
        break

In [ ]:
#[10323.0, 10367.0, 10419.0]

In [ ]:
for st in m.Lv1_L:
    if st.start.TmIdx == 10323:
        print(st.start.V, st.peak.V, st.mp[st.pp], st.ML[st.mp[st.pp] - 1].peak, st.ML[st.mp[st.pp]], m.Lv1_L.index(st))
        break

In [ ]:
[[st.start.V, st.peak.V] for st in m.Lv0_L[1520:1525]]
    

In [ ]:
[st for st in m.Lv3_L if s_idx<st.peak.TmIdx<=13000]

In [ ]:
[[pp.TmS, pp.drt, pp.ccHL[3],pp.ccHL[2]] for pp in m.PLv1_L 
             if pp.chain_layer == 0 and s_idx<pp.ML[pp.index[0]].peak.TmIdx<11500]

In [ ]:
m.Lv0_L[0].start

In [ ]:
pp = m.PLv0_L[4]

In [ ]:
[( (pp.ML[pp.index[0]].peak.TmIdx, pp.ccHL[2]), 
  pp.ML[pp.index[-1]].peak.TmIdx - pp.ML[pp.index[0]].peak.TmIdx,
  pp.ccHL[3] - pp.ccHL[2]) 
 for pp in m.PLv0_L[1:5]]

In [ ]:
dt_df.iloc[168].name

In [ ]:
fig, ax = plt.subplots(figsize=(1200/72,480/72))

# format the ticks
tmLabel = dt_df[80:220].index
x_idx = dt_df[80:220].TmIdx.values.tolist()
x_fmt = [x.strftime("%m/%d %H") for x in tmLabel]

def format_date(x,pos=None):   
    x = int(x)
    if x<0:
        return ''
    else:
        x2 = x_idx.index(x) -x_idx[0]
    return x_fmt[x2]

ax.xaxis.set_major_formatter(ticker.FuncFormatter(format_date))
ax.xaxis.set_major_locator(ticker.MultipleLocator(6))

# get xy data
x = data_up.TmIdx.values
ymin = data_up.open.values
ymax = data_up.close.values
ax.vlines(x, ymin, ymax, color='r', lw=5)
ax.vlines(x, data_up.low.values, data_up.high.values, color='r', lw=1)
x = data_down.TmIdx.values
ymin = data_down.close.values
ymax = data_down.open.values
ax.vlines(x, ymin, ymax, color='g', lw=5)
ax.vlines(x, data_down.low.values, data_down.high.values, color='g', lw=1)

#x = [d[0] for d in dd[:10]]
#y = [d[1] for d in dd[:10]]
#ax.plot(x, y, color='b')


fig.autofmt_xdate()
plt.savefig("image01.png")
#plt.show()

In [ ]:
def candel_plotter(ax, data_df, i, w):
    data = data_df[i-w:i+w]
    data_up = data.query('open <= close')
    data_down = data.query('open > close')
    
    x = data_up.TmIdx.values
    ax.vlines(x, data_up.open.values, data_up.close.values, color='r', lw=5)
    ax.vlines(x, data_up.low.values, data_up.high.values, color='r', lw=1)
    x = data_down.TmIdx.values
    ax.vlines(x, data_down.close.values, data_down.open.values, color='g', lw=5)
    ax.vlines(x, data_down.low.values, data_down.high.values, color='g', lw=1)
    
    #out = ax.plot(data1, data2, **param_dict)
    return None

In [ ]:
i, w = 10000,50
fig, ax = plt.subplots(1, 1,figsize=(1200/72,480/72))

# format the ticks
tmLabel = dt_df[i-w-int(w*0.3):i+w+int(w*0.3)].index
x_idx = dt_df[i-w-int(w*0.3):i+w+int(w*0.3)].TmIdx.values.tolist()
x_fmt = [x.strftime("%m/%d %H") for x in tmLabel]

def format_date(x,pos=None):   
    x = int(x)
    if x<0:
        return ''
    else:
        x2 = x_idx.index(x)
    return x_fmt[x2]

ax.xaxis.set_major_formatter(ticker.FuncFormatter(format_date))
ax.xaxis.set_major_locator(ticker.MultipleLocator(6))


candel_plotter(ax, dt_df, i, w)
fig.autofmt_xdate()
plt.show()

In [ ]:
x_fmt[18]

In [ ]:
dt_df[500:501].index

In [ ]:
dt_df.loc['2019-11-8 13':'2019-11-11']

In [ ]:
#dt_df.loc[:'1/3/2020', 'close']
#dt_df.loc[:datetime(2020,1,3), 'close']

In [ ]:
pd.core.index

In [ ]:
def verify_end_to_end(lv_L):
    # peak with start
    dd = [[st1.start.TmIdx, st1.peak, st2.start] for st1,st2 in zip(lv_L[:-2], lv_L[1:]) if st1.peak.TmIdx != st2.start.TmIdx]
    if len(dd) == 0:
        return True
    else:
        return dd

In [ ]:
def verity_valley_after_peak(lv_L):
    return True

In [ ]:
verify_end_to_end(m.Lv1_L)

### Trend Analysis

In [ ]:
dd = [[st.start.TmIdx, st.start.V, st.drt,st.peak.TmIdx, st.end.TmIdx,st.peak.TmIdx - st.start.TmIdx, 
      len(st.ES_stack), st.mp, st.mp[0]] for st in m.Lv1_L]

In [ ]:
df_out = pd.DataFrame(dd)
df_out.columns = ['TmS','start','drt', 'TmP', 'TmE', 'T', 'ES', 'mp', 'mp0']
df_out['A'] = -df_out.start.diff(-1)
df_out['st_cnt'] = -df_out.mp0.diff(-1)

In [ ]:
df_out.to_excel('data_csv\\lv1.xlsx')

In [ ]:
df_out.A.abs().quantile([.1, .15, .25,.5, .75,.85,.9])

In [ ]:
df_out.A.abs().mean()

In [ ]:
ss = df_out['A'].tolist()
ss.append(np.nan)
ss.append(np.nan)
df_out.loc[:,'A_next2'] = ss[2:]
df_out.loc[:,'A_next'] = ss[1:-1]

In [ ]:
df_out.head()

In [ ]:
df_out.loc[df_out['A']>0.5, [ 'A_next2']].quantile([.1, .15, .25,.3, .35,.4,.45,.5,.6, .7, .75,.8, .85,.9])

In [ ]:
df_out.loc[df_out['A']>0.7, ['A_next', 'A_next2']].quantile([.1, .15, .25,.3, .35,.4,.45,.5,.6, .7, .75,.8, .85,.9])

In [ ]:
df_out.query('A>0.7 and A_next<-0.6').A_next2.quantile([.1, .15, .25,.3, .35,.4,.45,.5,.6, .7, .75,.8, .85,.9])

In [ ]:
df_out.query('A>0.7 and A_next<-0.6 and A > -A_next')

In [ ]:
df_out.loc[df_out['A']>0.3, [ 'A_next','A_next2']].quantile([.1, .15, .25,.3, .35,.4,.45,.5,.6, .7, .75,.8, .85,.9])

In [ ]:
tmIdx = 15929


In [ ]:
# find Lv
def find_lv(tmIdx, level):
    global m
    lv_idx = [st.start.TmIdx for st in m.findList('st', level)]
    lv = m.findList('st', level)
    
    i = 0
    for tm in lv_idx[1:]:
        if tmIdx >= tm:
            i += 1
        else:
            break
    return i,lv[i]

In [ ]:
find_lv(tmIdx,0), find_lv(tmIdx, 1),find_lv(tmIdx, 2)

In [ ]:
# find Pair
def find_pair(tmIdx, level):
    global m
    idx = [p.TmS for p in m.findList('pair', level)]
    ll = m.findList('pair', level)
    
    i = 0
    for tm in idx[1:]:
        if tmIdx >= tm:
            i += 1
        else:
            break
    return i, ll[i]

In [ ]:
find_pair(tmIdx, 1).__dict__

In [ ]:
lv[0].__dict__

In [ ]:
m.Lv0_L[7]

In [ ]:
m.Lv1_L[3]

In [ ]:
m.Lv1_L[292].peak,m.Lv1_L[292],m.Lv1_L[292].end

In [ ]:
m.Lv0_L[3].__dict__

In [ ]:
len(m.PLv1_L)

In [ ]:
len(m.position)

In [ ]:
dd = [[op.drt,op.TmSig, op.TmOp, op.openV,op.pattern, op.pattern_objs] for op in m.position]

In [ ]:
df = pd.DataFrame(dd)

df.to_excel('position.xlsx')

In [ ]:
m.PCLv0_L[0].cL

In [ ]:
# print Pair list
dd = m.PLv1_L[1].calFeatures()
dd['TmS'] = []
dd['chain_layer'] = []
dd['index'] = []
dd['S'] =[]
dd['P'] = []
for k in dd.keys():
    dd[k] = []
for p in m.PLv1_L:
    print(p)
    r = p.calFeatures()
    r['TmS'] = p.TmS
    r['chain_layer'] = p.chain_layer
    r['index'] = p.index
    r['S'] = p.S
    r['P'] = p.P
    for k in r.keys():
        dd[k].append(r[k])
df = pd.DataFrame(dd)
df = df[['TmS','chain_layer',  'index', 'drt','is_flag','cl', 'ch', 'a','k','c_c_level', 'c_drt'
         ,'S', 'P',  'b_tm', 'b_w', 'c_a',  'cc',
        'p_v', 's_tm', 's_v']]
df.to_excel('PairLv1.xlsx')

In [ ]:
[p.index for p in m.PLv1_L]

In [ ]:
[st.distr() for st in m.Lv1_L[5].stick_stack]

In [ ]:
m.Lv1_L[1].stick_stack[0]

In [ ]:
m.Lv1_L[1].stick_stack[0].peak

In [ ]:
m.Lv1_L[1].stick_stack[0].distr()

In [ ]:
l = [[st.drt,st.amp()[0], st.amp()[1]] for st in m.Lv1_L]
df = pd.DataFrame(l)
df.to_excel('00lv_amp_by_st.xlsx')

In [ ]:
tt = [list(st.distr()[0]) + list(st.distr()[1]) for st in m.Lv0_L[132:135]]
tt_df = pd.DataFrame(tt)
tt_df.to_excel('stdistr.xlsx')

In [ ]:
x = m.PLv0_L[15].calFeatures()
op = Operator(x,3,'pair_drt')
op2 = Operator(x,0.5,'concentrated')
r = Node([op,op2],1)
r.do()

In [ ]:
EventFactory.event_config

In [ ]:
m.findList('pair',0).__name__

In [ ]:
d = {'level_num':0, 'obj_name': 'Stick', 'event_name':'NEW', 'obj_p':'m.position[0]', 'method':'check_event', 'param':''}
EventFactory.reg_event(d)

In [ ]:
EventFactory.remove_event(d)

In [ ]:
# print Pair Lv0
dd = m.PLv0_L[5].output()
for k in dd.keys():
    dd[k] = []
for p in m.PLv0_L:
    r = p.output()
    for k in r.keys():
        dd[k].append(r[k])
df = pd.DataFrame(dd)
df.to_excel('pair0.xlsx')

In [ ]:
# 把Lv0打印出来
dd = [[st.drt,st.start.TmIdx,st.start.V,st.peak.TmIdx, st.peak.V] for st in m.Lv0_L]
df_dd = pd.DataFrame(dd)
df_dd.to_excel('Lv0_L.xlsx')

In [ ]:
# 把Lv1打印出来
dd = [[st.drt,st.start.TmIdx,st.start.V,st.peak.TmIdx, st.peak.V,st.pp,st.mp] for st in m.Lv1_L]
df_dd = pd.DataFrame(dd)
df_dd.to_excel('Lv1_L.xlsx')

In [ ]:
# 把Lv2打印出来
dd = [[st.drt,st.start.TmIdx,st.start.V,st.peak.TmIdx, st.peak.V,st.pp,st.mp] for st in m.Lv2_L]
df_dd = pd.DataFrame(dd)
df_dd.to_excel('Lv2_L.xlsx')

In [ ]:
st = m.Lv0_L[3]
k_L = [[k[1],k[2]] for k in dt if k[4]>= st.start.TmIdx and k[4]<= st.peak.TmIdx]

interval = (st.peak.V - st.start.V)*st.drt/ 20
L = min(st.peak.V, st.start.V)
H = max(st.peak.V, st.start.V)
scale = [L]
for i in list(range(1,20)):
    scale.append(scale[i-1]+interval)
scale.append(H)

distr = np.zeros(20)
for k in k_L:
    distr_k = np.zeros(20)
    for i in list(range(20)):
        if (min(k) <= scale[i] <= max(k)) or (min(k) <= scale[i+1] <= max(k)):
            distr_k[i] = 1
    cnt = distr_k.sum()
    distr_k = distr_k/cnt
    distr = distr + distr_k
distr, distr.argmax()

In [ ]:
ll = np.zeros(20)
ll[2:4] = 1
cnt = ll.sum()
ll = ll/cnt

ll2 = np.ones(20)
lr = ll +ll2
lr

In [ ]:
a = [1,2,3]
a[:-1]

### draw and verified

In [ ]:
data = {
    'times': x['TmIdx'],
    'data': 
}

In [ ]:
# 绘图用插值函数
# input: 
#       1 - TmIdx array 
#       2 - Values array
# output: 按x轴查完值的value array
def interpoint(TmIdx_array, V_array):
    global x
    x['TmIdx'].astype('int64')
    print(len(x))
    df_tmp = pd.DataFrame({'V': V_array, 'TmIdx': TmIdx_array})
    df_tmp['TmIdx'].astype('int64')
    start_tm = int(TmIdx_array[0])
    end_tm = int(TmIdx_array[-1]) +1
    df_tmp = df_tmp.merge(x.iloc[start_tm:end_tm,:], how='right', left_on='TmIdx', right_on='TmIdx')
    df_tmp.sort_values(['TmIdx'], inplace=True)
    df_tmp['V'].interpolate(inplace=True)
    return df_tmp['V'].values.tolist(), df_tmp[['TmIdx','V']]

In [ ]:
def draw(lv_list):
    global dt, interpoint
    x_idx = [k[4] for k in dt]
    oringe_dt = [[k[0], k[3], k[2], k[1]] for k in dt]

    n = len(lv_list)
    drawing_line = []
    
    for i in list(range(n)):
        tm, v = [], []
        for stick in lv_list[i]:
            tm.append(stick.start.TmIdx)
            v.append(stick.start.V)
        dt_line = interpoint(tm,v)
        line = Line('stick')
        line.add('xxx', [k[4] for k in dt], dt_line
        ,is_datazoom_show=True
        )  
        drawing_line.append(line)
        
    overlap = Overlap()
    for line in drawing_line:
        overlap.add(line)
    
    return overlap
            

In [ ]:
draw([Stick.lv_L, TrendLv1.lv_L, TrendLv2.lv_L])

### 定点k-line

In [ ]:
def draw_k(center_point, half_window, stock_name='xxx'):
    line = Kline('stick')
    global dt, x
    
    dt_x = x.iloc[center_point - half_window//2:center_point + half_window//2,2].values.tolist()
    dt_tmp = [dd[:4] for dd in dt[center_point - half_window//2:center_point + half_window//2 ]]
    dt_line = [[k[0], k[3], k[2], k[1]] for k in dt_tmp]
    line.add(stock_name, dt_x, dt_line)  
    return line


In [ ]:
draw_k(15027,500)

In [ ]:
line = (Line()
       .add_yaxis()
       )

In [ ]:
_,p_df = interpoint([int(point1.TmIdx), int(point2.TmIdx)], [point1.V, point2.V])

### draw Pair

In [ ]:
p = find_pair(tmIdx,0)[1]
p.__dict__

In [ ]:
level = 1
center_point = tmIdx
half_window = 15
start = int(center_point - half_window)
end = int(center_point + half_window)
dt_x = x.iloc[center_point - half_window//2:center_point + half_window//2,2].values.tolist()

In [ ]:
point1 = m.findList('st',level-1)[p.index[0]].start
point1

In [ ]:
point2 = m.findList('st',level-1)[p.index[-1]].peak
point2

In [ ]:
_,df_p = interpoint([int(point1.TmIdx), int(point2.TmIdx)], [point1.V, point2.V])

In [ ]:
df_draw = x.iloc[start:end,].copy()
df_draw = df_draw.merge(df_p, how='left', left_on='TmIdx', right_on='TmIdx')
df_draw = df_draw.merge(dt_df[['open','high','low','close','TmIdx']], how='left', left_on='TmIdx', right_on='TmIdx')
df_draw.query('TmIdx>15900 and TmIdx<15940')

In [ ]:
df_p

In [ ]:
df_draw.fillna('', inplace=True)
df_draw

In [ ]:
import pyecharts.options as opts
COLOR = ['red', 'yellow', 'green']

In [ ]:
kl = (Kline()
      .add_xaxis(df_draw.date_str.tolist())
      .add_yaxis('pair', df_draw[['open','close','low','high']].values.tolist())
)
kl.render_notebook()

In [ ]:
l = (
    Line()
    .add_xaxis(df_draw.date_str.tolist())
    .add_yaxis("pair"
               , df_draw.V.tolist()
               ,is_symbol_show = False
               .linestyle_opts = opts.LineStyleOpts(color=color[level], width=4, opacity=0.6)
               
              )
)
l.render_notebook()

In [ ]:
kl.overlap(l).render_notebook()

In [ ]:
find_pair(14952,1)[1].__dict__

In [ ]:
draw_k(16027,500)

In [ ]:
x.loc[x['TmIdx']==14705,'datetime']

In [ ]:
x.loc[x['datetime']=='2020-02-06 14:00','TmIdx']

### draw scene

In [ ]:
m.Lv0_L[0]

In [ ]:
m.Lv1_L[2]

In [ ]:
m.findList('st', 0)[2]

In [ ]:
Lv2_idx = [st.start.TmIdx for st in m.Lv2_L]

In [ ]:
Lv2_idx = []
Lv2_idx = [st.start.TmIdx for st in m.findList('st', 2)]

In [ ]:
Lv2_idx[:10]

In [ ]:
def find_index(ll, num):
    i = 0
    for n in ll[1:]:
        if num >= n:
            i += 1
        else:
            break
    return i

        
Lv2_idx.index(440)

In [ ]:
idx = find_index(Lv2_idx, 9508)

In [ ]:
m.Lv2_L[idx].drt, m.Lv2_L[idx].start, m.Lv2_L[idx+1].start

In [ ]:
idx

In [ ]:
len(m.Lv2_L)

In [ ]:
i =0
for st in m.Lv2_L:
    print(i, st.start,st.pp)
    i += 1

In [ ]:
m.Lv2_L[66].stick_stack, m.Lv2_L[66].mp

In [ ]:
m.Lv2_L[67].stick_stack, m.Lv2_L[67].mp, m.Lv2_L[67].status

In [ ]:
CenterStrict(Stick.lv_L[1:], market)

In [ ]:
for c in market.CenterStrict_LD[10]:
    print(c)

In [ ]:
a = [1,2,3,4,9]
l = [1,2,3,4,9]
for aa,ll in zip(*[a,l]):
    print(aa,ll)

In [ ]:
m.__class__.__name__

In [ ]:
a = []
a.append([])
a.append([])
a.append([])

In [ ]:
a

In [ ]:
tmp = type('Xb', (object,), dict(a=1))
tmp = type('Xa', (object,), dict(a=1))

In [ ]:
a = [1,2,3]
b = [3,4,5]
for aa in a [1:
             :-1]:
    print(aa)

In [ ]:
d1 = {'a': 1}
d2 = {'b': 2}

In [ ]:
d1.append(d2)